# Links
*   [data.world](https://data.world/apnorc/ap-votecast-public-use-file)
*   [CSV](https://data.world/apnorc/ap-votecast-public-use-file/workspace/file?filename=AP_VOTECAST_2018_DATA.csv)
*   [Official VoteCast Codebook](https://data.world/apnorc/ap-votecast-public-use-file/workspace/file?filename=VC_PUF_codebook_final.pdf)
*   [Questionnaire](https://data.world/apnorc/ap-votecast-public-use-file/workspace/file?filename=VoteCast+Questionnaire.pdf)


---


*   [Project Codebook](https://docs.google.com/spreadsheets/d/1tJyMw_8Py4XLJ4PPAEfLMnwwumlNZ3IcuVw74XBLQ7A/edit?usp=sharing)


# Import Packages & Data

In [2]:
!pip install pandasql
!pip install chart-studio


  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=5ca709ca1610560047e73070df45ba873c190671afa93f602ffbe2da0c9ce74f
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql
     |████████████████████████████████| 81kB 2.2MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inspect
import pandasql
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import time
import chart_studio.dashboard_objs as dashboard

import IPython.display
from IPython.display import Image
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px

In [4]:
# pull data from data.world
#full_data = pd.read_csv('https://download.data.world/s/nyzwf5llkeokpvczdjsbfel7yt5zvu', header=0)
full_data = pd.read_csv('https://download.data.world/s/6ed2jwgz4kzqm46vb32gapbrhhushx', header=0)
print(full_data.shape)
print(full_data.head(30).to_string())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (12,15,16,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,40,41,42,43,44,45,46,47,48,69,70,71,72,73,74,75,76,77,78,79,80,81,83,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,209) have mixed types. Specify dtype option on import or set low_memory=False.



(138929, 220)
       su_id  pollclose_state_weight  pollclose_national_weight  finalvote_state_weight  finalvote_national_weight       mode             p_state            statenum                                    lva                         lvb                         lv       likelyvoter  wva                                                wvb            timevote                  race1_vote     race1_party                     race1_raceid     race1_officeid      race1_racetypeid              race2_vote       race2_party                             race2_raceid      race2_officeid      race2_racetypeid                race3_vote     race3_party                       race3_raceid race3_officeid      race3_racetypeid                 race4_vote   race4_party                                       race4_raceid                race4_officeid      race4_racetypeid                        race5_vote     race5_party                                       race5_raceid                race5_officeid 

# Clean-Up Functions

In [0]:
def to_binary(field, val_1):
  val_1 = int(val_1)
  field = str(field)
  field_0 = int(round(float(field.replace('(','').split(') ')[0]),0))
  #field_1 = field.replace('(','').split(') ')[1]
  if field_0 == val_1:
    choice = 1
  else:
    choice = 0
  return(choice)

def call_to_binary(frame, field, rename='', val_1=1):
  frame[field] = frame[field].apply(lambda x,val_1=val_1:to_binary(x,val_1) if pd.notnull(x) else x)
  if rename != '':
    frame = frame.rename(columns={field:rename})
  return(frame)

In [0]:
def to_ranked_reverse(field, n_choice):
  n_choice = int(n_choice)
  field = str(field)
  field_0 = int(round(float(field.replace('(','').split(') ')[0]),0))
  #field_1 = field.replace('(','').split(') ')[1]
  choice = field_0*-1+n_choice
  return(choice)

def call_to_ranked_reverse(frame, field, rename='',n_choice=''):
  if n_choice == '':
    n_choice = str(frame[field].nunique())
  frame[field] = frame[field].apply(lambda x,n_choice=n_choice:to_ranked_reverse(x,n_choice) if pd.notnull(x) else x)
  if rename != '':
    frame = frame.rename(columns={field:rename})
  return(frame)

In [0]:
def remove_code(field):
  field = str(field)
  field = field.split(') ')[1]
  return(field)

def call_remove_code(frame, field, rename=''):
  frame[field] = frame[field].apply(lambda x:remove_code(x) if pd.notnull(x) else x)
  if rename != '':
    frame = frame.rename(columns={field:rename})
  return(frame)

In [0]:
def insert_dummies(frame, field, pos):
  frame = pd.DataFrame(frame)
  dummies = pd.get_dummies(frame[field], dummy_na=True)
  dummies.columns = dummies.columns.fillna('() NaN')
  if dummies.sum(axis=0)[-1] == 0:
    dummies = dummies.drop(columns={'() NaN'})
  col_names = dummies.columns
  for i in range(dummies.shape[1]):
    new_col = str(field+'_'+col_names[i].split(') ')[1]).lower().replace('the','').replace(' ','').replace(',','').replace('/','-').replace('$','').replace('.','').replace('\'','').replace('-','_')
    dummies = dummies.rename(columns={col_names[i]:str(new_col)})
    pos = pos+1
    vals = list(dummies[new_col])
    frame.insert(pos, new_col, vals)
  frame = frame.drop(columns={field})
  return(frame)

def call_insert_dummies(frame, field, rename=''):
  if rename != '':
    frame = frame.rename(columns={field:rename})
    field = rename
  frame = insert_dummies(frame,field,frame.columns.get_loc(field))
  return(frame)

In [0]:
def to_3_choice_flip(field, val_2, val_0):
  val_2 = int(val_2)
  val_0 = int(val_0)
  field = str(field)
  field_0 = int(round(float(field.replace('(','').split(') ')[0]),0))
  #field_1 = field.replace('(','').split(') ')[1]
  if field_0 == val_2:
    choice = 2
  elif field_0 == val_0:
    choice = 0
  else:
    choice = 1
  return(choice)

def call_to_3_choice_flip(frame, field, rename='', val_2=1,val_0=2):
  frame[field] = frame[field].apply(lambda x,val_2=val_2,val_0=val_0:to_3_choice_flip(x,val_2,val_0) if pd.notnull(x) else x)
  if rename != '':
    frame = frame.rename(columns={field:rename})
  return(frame)

In [0]:
def to_prob(field):
  field = str(field)
  field_0 = float(field.replace('(','').split(') ')[0])*0.1
  if field_0 > 1.0:
    field_0 = 1.0
  return(field_0)

def call_to_prob(frame, field, rename=''):
  frame[field] = frame[field].apply(lambda x:to_prob(x) if pd.notnull(x) else x)
  if rename != '':
    frame = frame.rename(columns={field:rename})
  return(frame)

In [0]:
def combo_raceth(re, re5):
  try:
    re = str(re)
    re_0 = int(round(float(re.replace('(','').split(') ')[0]),0))
  except:
    re_0 = -1
  try:
    re5 = str(re5)
    re5_0 = int(round(float(re5.replace('(','').split(') ')[0]),0))
  except:
    re5_0 = -1

  if re_0 == 1 or re5_0 == 1:
    val = '(1) wht'
  elif re_0 == 2 or re5_0 == 2:
    val = '(2) blk'
  elif re_0 == 3 or re5_0 == 3:
    val = '(3) hsp'
  elif re_0 == 4 or re5_0 == 4:
    val = '(4) asn'
  elif re_0 == 5:
    val = '(5) ntvam'
  elif re_0 == 6:
    val = '(6) hipac'
  elif re_0 == 7 or re5_0 == 5:
    val = '(7) othr'
  else:
    val = None
  return(val)

def call_combo_raceth(frame):
  frame['raceth'] = frame.apply(lambda x:combo_raceth(x['raceth'], x['raceth5']), axis=1)
  frame = call_insert_dummies(frame,'raceth')
  frame = frame.drop(columns={'raceth5'})
  return(frame)

In [0]:
def combo_relig(rlg, rlg4):
  try:
    rlg = str(rlg)
    rlg_0 = int(round(float(rlg.replace('(','').split(') ')[0]),0))
  except:
    rlg_0 = -1
  try:
    rlg4 = str(re5)
    rlg4_0 = int(round(float(rlg4.replace('(','').split(') ')[0]),0))
  except:
    rlg4_0 = -1

  if rlg_0 == 1 or rlg_0 == 4 or rlg4_0 == 1:
    val = '(1) prot_chrst'
  elif rlg_0 == 2 or rlg4_0 == 2:
    val = '(2) catholic'
  elif rlg_0 == 3:
    val = '(3) mormon'
  elif rlg_0 == 5:
    val = '(5) jewish'
  elif rlg_0 == 6:
    val = '(6) muslim'
  elif rlg_0 == 7 or rlg4_0 == 3:
    val = '(7) othr'
  elif rlg_0 == 8 or rlg4_0 == 4:
    val = '(8) none'
  else:
    val = None
  return(val)

def call_combo_relig(frame):
  frame['relig'] = frame.apply(lambda x:combo_relig(x['relig'], x['relig4']), axis=1)
  frame = call_insert_dummies(frame,'relig')
  frame = frame.drop(columns={'relig4'})
  return(frame)

In [0]:
def combo_wv(a, b):
  try:
    a = str(a)
    a_0 = int(round(float(a.replace('(','').split(') ')[0]),0))
  except:
    a_0 = -1
  try:
    b = str(b)
    b_0 = int(round(float(b.replace('(','').split(') ')[0]),0))
  except:
    b_0 = -1

  if a_0 == 1 or b_0 == 1:
    val = '(1) election day'
  elif a_0 == 2 or b_0 == 2:
    val = '(2) early'
  else:
    val = None
  return(val)

def call_combo_wv(frame):
  frame['wva'] = frame.apply(lambda x:combo_wv(x['wva'], x['wvb']), axis=1)
  frame = call_to_binary(frame,'wva','whenvote18_dayof')
  frame = frame.drop(columns={'wvb'})
  return(frame)

In [0]:
def combo_party(p, pf):
  try:
    p = str(p)
    p_0 = int(round(float(p.replace('(','').split(') ')[0]),0))
  except:
    p_0 = -1
  try:
    pf = str(pf)
    pf_0 = int(round(float(pf.replace('(','').split(') ')[0]),0))
  except:
    pf_0 = -1

  if p_0 == 1:
    val = 0
  elif p_0 == 2:
    val = 4
  elif pf_0 == 1:
    val = 1
  elif pf_0 == 2:
    val = 3
  elif pf_0 == 3:
    val = 2
  else:
    val = None
  return(val)

def call_combo_party(frame):
  frame['party'] = frame.apply(lambda x:combo_party(x['party'], x['partyfull']), axis=1)
  frame = frame.drop(columns={'partyfull'})
  return(frame)

In [0]:
def ideo(i):
  i = str(i)
  i_0 = int(round(float(i.replace('(','').split(') ')[0]),0))
  val = i_0-1
  return(val)

def call_ideo(frame):
  frame['ideo'] = frame['ideo'].apply(lambda x:ideo(x) if pd.notnull(x) else x)
  return(frame)

In [0]:
def churchattend(ca):
  ca = str(ca)
  ca_0 = int(round(float(ca.replace('(','').split(') ')[0]),0))

  if ca_0 == 1:
    val = 0
  elif ca_0 == 2:
    val = 3
  elif ca_0 == 3:
    val = 12
  elif ca_0 == 4:
    val = 36
  elif ca_0 == 5:
    val = 52
  else:
    val = None
  return(val)

def call_churchattend(frame):
  frame['attendance'] = frame['attendance'].apply(lambda x:churchattend(x) if pd.notnull(x) else x)
  frame = frame.rename(columns={'attendance':'churchattend'})
  return(frame)

In [0]:
def suppTrump(n,v16,_v16,v20,_v20,appr,vimp,fav):
  agg = 0
  nones = 0
  if int(_v16) == 0:
    agg += int(v16)
  else:
    nones += 1
  if int(_v20) == 0:
    agg += int(v20)
  else:
    nones += 1
  try:
    appr = int(appr)
    if appr >= 2:
      agg += 1
  except:
    nones += 1
  try:
    vimp = int(vimp)
    if vimp == 2:
      agg += 1
  except:
    nones += 1
  try:
    fav = int(fav)
    if fav >= 2:
      agg += 1
  except:
    nones += 1

  if nones == n:
    val = None
  elif agg >= 1:
    val = 1
  else:
    val = 0
  return(val)

def call_suppTrump(frame):
  n = len(inspect.signature(suppTrump).parameters)-3
  frame['suppTrump'] = frame.apply(lambda x,n=n:suppTrump(n,x['vote16_donaldtrump'],x['vote16_nan'],x['vote20_donaldtrumprepublican'],x['vote20_nan'],x['apprTrump'],x['trumpvoteimpact'],x['favTrump']), axis=1)
  frame = frame.drop(columns={'voted16','vote16_hillaryclinton','vote16_someoneelse','vote20_donaldtrumprepublican','vote20_democraticcandidate','vote20_itdepends','vote20_wouldnotvote','vote20_nan','apprTrump','trumpvoteimpact','favTrump'}) #comment out line to inspect elements in test cases
  return(frame)

# Munging

### Test Functions

In [0]:
# test function
#df_test = full_data.copy(deep=True).replace({'(99) DON’T KNOW/SKIPPED/REFUSED (VOL)':np.NaN}).replace({'(99) DON\'T KNOW/SKIPPED/REFUSED (VOL)':np.NaN})

#field = 'lv'
#rename = 'P_vote'
#val_2 = '1'
#val_0 = '2'
#print(df_test[field].unique())
#print(df_test.columns.get_loc(field))

# to binary
#df_test[str('_'+field)] = df_test[field].apply(lambda x,val_0='2':to_binary(x,val_0) if pd.notnull(x) else x)
#df_test = call_to_binary(df_test,field,'2')

# to ranked reverse
#df_test[str('_'+field)] = df_test[field].apply(lambda x,n_choice=str(df_test[field].nunique()):to_ranked_reverse(x,n_choice) if pd.notnull(x) else x)
#df_test = call_to_ranked_reverse(df_test,field,rename)
#df_test = call_to_ranked_reverse(df_test,'healthlaw','supACA')


# remove code
#df_test[str('_'+field)] = df_test[field].apply(lambda x:remove_code(x) if pd.notnull(x) else x)
#df_test = call_remove_code(df_test,field,rename)

# insert dummies
#df_test = insert_dummies(df_test, field, df_test.columns.get_loc(field))
#df_test = call_insert_dummies(df_test,field)

# to 3 choice flip
#df_test[str('_'+field)] = df_test[field].apply(lambda x,val_2=val_2,val_0=val_0:to_3_choice_flip(x,val_2,val_0) if pd.notnull(x) else x)
#df_test = call_to_3_choice_flip(df_test,field,rename,'2','1')

# to prob
#df_test[str('_'+field)] = df_test[field].apply(lambda x:to_prob(x) if pd.notnull(x) else x)
#df_test = call_to_prob(df_test,field,rename)
#print(df_test[['lv','_lv']].drop_duplicates())

# combo raceth
#df_test = call_combo_raceth(df_test)

# combo relig
#df_test = call_combo_relig(df_test)

# combo wv
#df_test = call_combo_wv(df_test)

# combo party
#df_test = call_combo_party(df_test)
#print(df_test[['party','partyfull','_party']].drop_duplicates())
# custom: ideo
#df_test = call_ideo(df_test)

# attendance
#df_test = call_churchattend(df_test)

#print(df_test['ideo'].head())

#print(df_test[str('_'+field)].max())
#print(df_test[[field,str('_'+field)]].dropna().head(20).to_string())
#print(df_test.head(20).to_string())

### Active Code

In [19]:
# rebase dataframe
df = full_data.copy(deep=True).replace({'(99) DON’T KNOW/SKIPPED/REFUSED (VOL)':np.NaN}).replace({'(99) DON\'T KNOW/SKIPPED/REFUSED (VOL)':np.NaN})

# su_id
df = df.drop(columns={'pollclose_state_weight','pollclose_national_weight','finalvote_state_weight','finalvote_national_weight','statenum','lvb'})
df = call_to_binary(df,'mode',val_1=2)
df = df.rename(columns={'p_state':'state'})
df = call_to_ranked_reverse(df,'lva','electinterest')
df = call_to_prob(df,'lv','P_vote')
df = call_to_binary(df,'likelyvoter')
df = call_combo_wv(df)
df = call_to_binary(df,'timevote')
df = call_remove_code(df,'race1_vote')
df = call_remove_code(df,'race1_party')
df = call_remove_code(df,'race2_vote')
df = call_remove_code(df,'race2_party')
df = call_remove_code(df,'race3_vote')
df = call_remove_code(df,'race3_party')
df = call_remove_code(df,'race4_vote')
df = call_remove_code(df,'race4_party')
df = call_remove_code(df,'race5_vote')
df = call_remove_code(df,'race5_party')
df = df.drop(columns={'race1_raceid','race1_officeid','race1_racetypeid','race2_raceid','race2_officeid','race2_racetypeid','race3_raceid',\
                      'race3_officeid','race3_racetypeid','race4_raceid','race4_officeid','race4_racetypeid','race5_raceid','race5_officeid',\
                      'race5_racetypeid','ballot1_vote','ballot1_party','ballot1_raceid','ballot1_officeid','ballot1_racetypeid','senvote',\
                      'senspvote','govvote','mainechoice2'})
df = call_to_binary(df,'track')
df = call_insert_dummies(df,'issues')
df = call_to_3_choice_flip(df,'life')
df = call_to_ranked_reverse(df,'presapp4','apprTrump')
df = call_to_3_choice_flip(df,'trumpfactor3','trumpvoteimpact')
df = call_to_binary(df,'trumptraitstemperament','trumpprestemp')
df = call_to_binary(df,'trumptraitschange','trumpchange')
df = call_to_binary(df,'trumptraitsleader','trumpleader')
df = call_to_binary(df,'trumptraitshonest','trumphonest')
df = call_to_binary(df,'trumptraitscares','trumpcares')
df = call_to_binary(df,'trumptraitsstandsup','trumpstandsup')
df = call_to_binary(df,'trumpjobecon','trumpecon')
df = call_to_binary(df,'trumpjobhealth','trumphealthcare')
df = call_to_binary(df,'trumpjobimmig','trumpimmigration')
df = call_to_binary(df,'trumpjobtrade','trumptrade')
df = call_to_binary(df,'trumpjobscotus','trumpscotus')
df = call_to_binary(df,'trumpjobborder','trumpborder')
df = call_to_ranked_reverse(df,'favtrump','favTrump')
df = call_to_ranked_reverse(df,'favrep','favR')
df = call_to_ranked_reverse(df,'favdem','favD')
df = call_to_ranked_reverse(df,'favsendem','favsenD')
df = call_to_ranked_reverse(df,'favsenrep','favsenR')
df = call_to_ranked_reverse(df,'favsenind','favsenI')
df = call_to_ranked_reverse(df,'favgovdem','favgovD')
df = call_to_ranked_reverse(df,'favgovrep','favgovR')
df = call_to_ranked_reverse(df,'favgovind','favgovI')
df = call_to_ranked_reverse(df,'favfeinstein','favFeinsteinD_CA')
df = call_to_ranked_reverse(df,'favdeleon','favDeLeonD_CA')
df = call_to_ranked_reverse(df,'favssenspecialdem','favsenD_MN')
df = call_to_ranked_reverse(df,'favssenspecialrep','favsenR_MN')
df = call_to_ranked_reverse(df,'favespy','favEspyD_MS')
df = call_to_ranked_reverse(df,'favhydesmith','favHydeSmithR_MS')
df = call_to_ranked_reverse(df,'favmcdaniel','favMcDanielR_MS')
df = call_to_ranked_reverse(df,'supremecourt','importantscotus')
df = call_to_ranked_reverse(df,'partycontrol','importantpartycontrol')
df = call_to_ranked_reverse(df,'congressjob4','apprCongress')
df = call_to_ranked_reverse(df,'trustgovt')
df = call_to_3_choice_flip(df,'ethics','trumpethics')
df = call_to_ranked_reverse(df,'govtangr','govtfeels')
df = call_to_binary(df,'govtdo','biggovt')
df = call_to_ranked_reverse(df,'nec','natecon')
df = call_to_ranked_reverse(df,'getahead','famfinancial')
df = call_to_3_choice_flip(df,'tradenationalecon','trumpnatecon')
df = call_to_3_choice_flip(df,'tradelocalecon','trumplocalecon')
df = call_to_3_choice_flip(df,'econfairwealthy')
df = call_to_3_choice_flip(df,'econfairmiddle')
df = call_to_3_choice_flip(df,'econfairpoor')
df = call_to_ranked_reverse(df,'taxcuts')
df = call_to_ranked_reverse(df,'healthlaw','supACA')
df = call_to_binary(df,'healthgov','biggovt_health')
df = call_to_binary(df,'immdeport','immpol')
df = call_to_binary(df,'immbetter','immhelp')
df = call_to_ranked_reverse(df,'immwall','borderwall')
df = call_to_3_choice_flip(df,'racerel','raceadvantage')
df = call_to_ranked_reverse(df,'climate')
df = call_to_3_choice_flip(df,'gunpolicy')
df = call_to_ranked_reverse(df,'abortion')
df = call_to_ranked_reverse(df,'metoobelieve')
df = call_to_ranked_reverse(df,'metoodefend')
df = call_to_binary(df,'pcspeech',val_1=2)
df = call_to_binary(df,'marijuana')
df = call_to_ranked_reverse(df,'opioid')
df = call_to_3_choice_flip(df,'safeterror','trumpterror')
df = call_to_3_choice_flip(df,'safecrime','trumpcrime')
df = call_to_3_choice_flip(df,'safecyber','trumpcyber')
df = call_to_binary(df,'russia','russiacollude16')
df = call_to_ranked_reverse(df,'interference','interference18')
df = call_insert_dummies(df,'q2020vote','vote20')
df = call_to_binary(df,'repintentions','goodintentR')
df = call_to_binary(df,'demintentions','goodintentD')
df = call_churchattend(df)
df = df.drop(columns={'married'})
df = call_to_binary(df,'married2','married')
df = call_insert_dummies(df,'gunowner')
df = call_insert_dummies(df,'union')
df = call_insert_dummies(df,'vet')
df = call_to_binary(df,'lgb')
df = call_to_binary(df,'transgender')
df = call_to_binary(df,'borncitizen')
df = call_to_ranked_reverse(df,'govjobkasich4','apprKasichR_OH')
df = call_to_ranked_reverse(df,'govjobsnyder4','apprSnyderR_MI')
df = call_to_ranked_reverse(df,'favsessions','favSessionsR_AL')
df = call_to_ranked_reverse(df,'favjones','favJonesD_AL')
df = call_to_ranked_reverse(df,'favpalin','favPalinR_AK')
df = call_to_ranked_reverse(df,'favflake','favFlakeR_AZ')
df = call_to_ranked_reverse(df,'favcotton','favCottonR_AR')
df = call_to_ranked_reverse(df,'favboozman','favBoozmanR_AR')
df = call_to_ranked_reverse(df,'favemanuel','favEmanuelD_IL')
df = call_to_ranked_reverse(df,'favpence','favPenceR_IN')
df = call_to_ranked_reverse(df,'favfranken','favFrankenD_MN')
df = call_to_ranked_reverse(df,'favinslee','favInsleeD_WA')
df = call_to_ranked_reverse(df,'medicaid','supmedicaid')
df = call_to_ranked_reverse(df,'sportsbetting','supsportbet')
df = call_to_ranked_reverse(df,'casinos','supcasinos')
df = call_to_3_choice_flip(df,'jobsit')
df = call_to_binary(df,'akdividends','oildiv_AK',2)
df = call_to_ranked_reverse(df,'deathpenalty','supdeathpen')
df = call_to_ranked_reverse(df,'affhousing','affordhousing')
df = call_to_ranked_reverse(df,'teacherstrikes','supteachstrike')
df = call_to_ranked_reverse(df,'secondchancelaw','sup2ndchancelaw')
df = call_to_ranked_reverse(df,'ilbudget','budget_IL')
df = call_to_ranked_reverse(df,'corruption','corrupt')
df = call_to_ranked_reverse(df,'teachersunions','supteachunion')
df = call_to_ranked_reverse(df,'edbudget','supedufunding')
df = call_to_ranked_reverse(df,'transportation','supinfrastructure')
df = call_to_binary(df,'prostitution','legalprost_NV')
df = call_insert_dummies(df,'govworker','govwork')
df = call_to_ranked_reverse(df,'incometax','supincometax_WA')
df = call_to_binary(df,'energy','altenergy',2)
df = call_insert_dummies(df,'coalhh','coalwork')
df = call_to_binary(df,'opioidexp')
df = call_to_3_choice_flip(df,'endangeredspecies','endangeredprotections','2','1')
df = call_to_ranked_reverse(df,'murkowskisc','apprMurkowskiR_AK_scotus')
df = call_to_ranked_reverse(df,'collinssc','apprCollinsR_ME_scotus')
df = call_to_3_choice_flip(df,'gastax','gastax_CA','2','1')
df = call_to_binary(df,'harrispres','presKHarris')
df = call_to_binary(df,'murphypres','presCMurphy')
df = call_to_binary(df,'bidenpres','presJBiden')
df = call_to_binary(df,'mapreswarren','presEWarren')
df = call_to_binary(df,'maprespatrick','presDPatrick')
df = call_to_binary(df,'nypresbloomberg','presMBloomberg')
df = call_to_binary(df,'nypresgillibrand','presKGillibrand')
df = call_to_binary(df,'klobucharpres','presAKlobuchar')
df = call_to_binary(df,'merkleypres','presJMerkley')
df = call_to_binary(df,'sanderspres','presBSanders')
df = call_to_binary(df,'unfairflgillum','unfairGillumD_FL')
df = call_to_binary(df,'unfairfldesantis','unfairDeSantisR_FL')
df = call_to_binary(df,'unfairnjmenendez','unfairMenendezD_NJ')
df = call_to_binary(df,'unfairnjhugin','unfairHuginR_NJ')
df = call_to_binary(df,'comarijuana','marijuana_CO')
df = call_to_ranked_reverse(df,'redtide','redtide_FL')
df = call_to_ranked_reverse(df,'firstblackgov','Abrams1stblackgov_GA')
df = call_to_ranked_reverse(df,'allowedvote','allowedvote_GA')
df = call_to_ranked_reverse(df,'easevoting')
df = call_to_ranked_reverse(df,'emergencyresponse','emrgncyresp')
df = call_insert_dummies(df,'tradedemvsrep','bettertrade')
df = call_to_binary(df,'kstax','tax_KS')
df = call_insert_dummies(df,'mshydesmithvsespy','MSmatchup1')
df = call_insert_dummies(df,'msmcdanielvsespy','MSmatchup2')
df = call_to_ranked_reverse(df,'preexistingcond','preexistcond')
df = call_to_ranked_reverse(df,'q2020excitement','excite20')
df = call_to_binary(df,'honestmenendez','honestMenendezD_NJ')
df = call_to_binary(df,'honesthugin','honestHuginR_NJ')
df = call_to_ranked_reverse(df,'taylor','favTSwift')
df = call_to_3_choice_flip(df,'confederateflag','opinconfflag')
df = call_to_3_choice_flip(df,'orourketooliberal','libORourkeD_TX')
df = call_to_binary(df,'cruzfocus','priorityCruzR_TX')
df = call_to_binary(df,'romneytrump','standupRomneyR_UT','1')
df = call_to_binary(df,'qpvvote','voted16',4)
df = call_insert_dummies(df,'q2016vote','vote16')
df = call_to_binary(df,'firsttime','firsttimevoter18')
df = call_to_binary(df,'qpvvote3','voted14',4)
df = call_insert_dummies(df,'wnv','wnv18')
df = call_to_binary(df,'breaka','violencespeechD')
df = call_to_binary(df,'breakb','violencespeechR')
df = call_insert_dummies(df,'breakc','votedecide18')
df = call_to_binary(df,'sex','sex_m')
df = call_insert_dummies(df,'age65','age')
df = call_combo_raceth(df)
df = call_insert_dummies(df,'cuban')
df = call_insert_dummies(df,'educ','edu')
df = call_insert_dummies(df,'income')
df = call_combo_party(df)
df = call_ideo(df)
df = call_combo_relig(df)
df = call_to_binary(df,'bornagain')
df = call_insert_dummies(df,'sizeplace')
df = df.drop(columns={'formflag'})

print(df.shape)
print(df.head(10).to_string())

(138929, 273)
      su_id  mode               state  electinterest  P_vote  likelyvoter  whenvote18_dayof  timevote         race1_vote race1_party race2_vote race2_party     race3_vote race3_party         race4_vote race4_party                race5_vote race5_party  track  issues_economyandjobs  issues_healthcare  issues_immigration  issues_taxes  issues_abortion  issues_gunpolicy  issues_environment  issues_foreignpolicy  issues_terrorism  issues_nan  life  apprTrump  trumpvoteimpact  trumpprestemp  trumpchange  trumpleader  trumphonest  trumpcares  trumpstandsup  trumpecon  trumphealthcare  trumpimmigration  trumptrade  trumpscotus  trumpborder  favTrump  favR  favD  favsenD  favsenR  favsenI  favgovD  favgovR  favgovI  favFeinsteinD_CA  favDeLeonD_CA  favsenD_MN  favsenR_MN  favEspyD_MS  favHydeSmithR_MS  favMcDanielR_MS  importantscotus  importantpartycontrol  apprCongress  trustgovt  trumpethics  govtfeels  biggovt  natecon  famfinancial  trumpnatecon  trumplocalecon  econfairweal

In [20]:
df_trump = call_suppTrump(df)
print("proportion with suppTrump not null:",df_trump['suppTrump'].count()/df_trump.shape[0])
df_trump = df_trump.dropna(subset=['suppTrump'])
print("proportion with suppTrump == 1:",df_trump['suppTrump'].mean())
df_trump = df_trump.drop(columns={'su_id','race1_vote','race1_party','race2_vote','race2_party','race3_vote','race3_party','race4_vote','race4_party','race5_vote','race5_party',\
                          'favFeinsteinD_CA','favDeLeonD_CA','favsenD_MN','favsenR_MN','favEspyD_MS','favHydeSmithR_MS','favMcDanielR_MS','apprKasichR_OH','apprSnyderR_MI',\
                          'favSessionsR_AL','favJonesD_AL','favPalinR_AK','favFlakeR_AZ','favCottonR_AR','favBoozmanR_AR','favEmanuelD_IL','favPenceR_IN','favFrankenD_MN',\
                          'favInsleeD_WA','oildiv_AK','budget_IL','legalprost_NV','supincometax_WA','apprMurkowskiR_AK_scotus','apprCollinsR_ME_scotus','gastax_CA','presKHarris',\
                          'presCMurphy','presJBiden','presEWarren','presDPatrick','presMBloomberg','presKGillibrand','presAKlobuchar','presJMerkley','presBSanders','unfairGillumD_FL',\
                          'unfairDeSantisR_FL','unfairMenendezD_NJ','unfairHuginR_NJ','marijuana_CO','redtide_FL','Abrams1stblackgov_GA','allowedvote_GA','tax_KS',\
                          'msmatchup1_cindyhyde_smith','msmatchup1_mikeespy','msmatchup1_undecided','msmatchup1_wouldnotvote','msmatchup1_nan','msmatchup2_chrismcdaniel',\
                          'msmatchup2_mikeespy','msmatchup2_undecided','msmatchup2_wouldnotvote','msmatchup2_nan','honestMenendezD_NJ','honestHuginR_NJ','favTSwift','libORourkeD_TX',\
                          'priorityCruzR_TX','standupRomneyR_UT','wnv18_idontlikecandidates','wnv18_idontlikepoliticsgenerally','wnv18_ittakestoomuchtimetovote',\
                          'wnv18_idontknowenoughaboutcandidatespositions','wnv18_myvotedoesntmatter','wnv18_idonthaverequiredidentification','wnv18_idontknowwheretovote',\
                          'wnv18_nan','votedecide18_iveknownallalong','votedecide18_idecidedovercourseofcampaign','votedecide18_idecidedinlastfewdays','votedecide18_istillhaventdecided',\
                          'votedecide18_nan'})
df_trump = df_trump[df_trump['issues_nan'] == 0]
df_trump = df_trump[df_trump['age_nan'] == 0]
df_trump = df_trump[df_trump['raceth_nan'] == 0]
df_trump = df_trump[df_trump['edu_nan'] == 0]
df_trump = df_trump[df_trump['income_nan'] == 0]
df_trump = df_trump[df_trump['relig_nan'] == 0]
df_trump = df_trump[df_trump['sizeplace_nan'] == 0]
df_trump = df_trump.drop(columns={'issues_nan','age_nan','raceth_nan','edu_nan','income_nan','relig_nan','sizeplace_nan',\
                          'gunowner_gunownerself','gunowner_guninhousehold','gunowner_no','gunowner_nan',\
                          'union_unionmemberself','union_unionmemberinhousehold','union_no','union_nan',\
                          'vet_veteranself','vet_veteraninhousehold','vet_no','vet_nan',\
                          'govwork_governmentworkerself','govwork_governmentworkerinhousehold','govwork_no','govwork_nan',\
                          'coalwork_coalindustryworkerself','coalwork_coalindustryworkerinhousehold','coalwork_no','coalwork_nan',\
                          'bettertrade_democraticparty','bettertrade_republicanparty','bettertrade_bothequally','bettertrade_nan',\
                          'cuban_cuban','cuban_orhispanic','cuban_non_hispanic','cuban_nan',\
                          'firsttimevoter18','favgovI','favsenI','supsportbet','sup2ndchancelaw'})
df_trump = df_trump.dropna(thresh=len(df_trump)-10000, axis=1)
#plt.hist(df_trump.isna().sum())
#plt.show()
df_trump = df_trump.dropna(axis=0)
print(df_trump.shape)
print(df_trump.describe().to_string())
#print(df_trump.head().to_string())

proportion with suppTrump not null: 0.9992658120334847
proportion with suppTrump == 1: 0.4276401564537158
(106855, 64)
                mode  electinterest         P_vote    likelyvoter       timevote          track  issues_economyandjobs  issues_healthcare  issues_immigration   issues_taxes  issues_abortion  issues_gunpolicy  issues_environment  issues_foreignpolicy  issues_terrorism   famfinancial   trumpnatecon  trumplocalecon         supACA         immpol  russiacollude16  vote16_donaldtrump     vote16_nan        voted14  violencespeechD  violencespeechR          sex_m      age_18_24      age_25_29      age_30_39      age_40_49      age_50_64        age_65+     raceth_wht     raceth_blk     raceth_hsp     raceth_asn   raceth_ntvam   raceth_hipac    raceth_othr  edu_highschoolorless  edu_somecollege_assocdegree  edu_collegegraduate  edu_postgraduatestudy  income_under25000  income_25000_49999  income_50000_74999  income_75000_99999  income_100000ormore          party           ideo  

# Modeling

## Estimating likelihood of supporting Trump

### Logistic Regression

In [21]:
df_l = df_trump.copy(deep=True).drop(columns={'state'})

X_context = df_l.drop(columns={'issues_terrorism','age_65+','raceth_othr','edu_postgraduatestudy','income_100000ormore','relig_othr','sizeplace_rural'})
y = df_l['suppTrump']

X_train_context, X_test_context, y_train, y_test = train_test_split(X_context, y, test_size=0.3, random_state=1)
X_train = X_train_context.drop(columns={'suppTrump','vote16_donaldtrump','vote16_nan'})
X_test = X_test_context.drop(columns={'suppTrump','vote16_donaldtrump','vote16_nan'})
X_train_context = X_train_context[['suppTrump','vote16_donaldtrump','vote16_nan']]
X_test_context = X_test_context[['suppTrump','vote16_donaldtrump','vote16_nan']]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(74798, 53)
(32057, 53)
(74798,)
(32057,)


In [22]:
X_train_intercept = X_train.copy(deep = True)
X_train_intercept['intercept'] = 1
logmodel_vars = sm.Logit(y_train,X_train_intercept)
result = logmodel_vars.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.180769
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:              suppTrump   No. Observations:                74798
Model:                          Logit   Df Residuals:                    74744
Method:                           MLE   Df Model:                           53
Date:                Fri, 20 Mar 2020   Pseudo R-squ.:                  0.7340
Time:                        13:17:01   Log-Likelihood:                -13521.
converged:                       True   LL-Null:                       -50836.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
mode                           -0.0456      0.082     -0.554      0.580   

In [23]:
time_0 = time.time()
logmodel = LogisticRegression(max_iter=300)
logmodel.fit(X_train, y_train)
time_1 = time.time()
pred_train = logmodel.predict(X_train)
time_2 = time.time()
pred_test = logmodel.predict(X_test)
time_3 = time.time()
print('Time to train:', time_1-time_0, 'seconds')
print('Time to predict train:', time_2-time_1, 'seconds')
print('Time to predict test:', time_3-time_2, 'seconds')
print('Total time:', time_3-time_0, 'seconds')
print('Train accuracy:',accuracy_score(y_train, pred_train))
print('\nConfusion matrix (suppTrump, prediction):\n',confusion_matrix(y_test, pred_test))
print('Test accuracy:',accuracy_score(y_test, pred_test))

Time to train: 2.2149834632873535 seconds
Time to predict train: 0.017848730087280273 seconds
Time to predict test: 0.007987737655639648 seconds
Total time: 2.2408199310302734 seconds
Train accuracy: 0.9308537661434798

Confusion matrix (suppTrump, prediction):
 [[17553  1045]
 [ 1220 12239]]
Test accuracy: 0.9293446049224818


### Random Forest

In [24]:
time_0 = time.time()
rfmodel = RandomForestClassifier(n_estimators=100, max_depth=15)
rfmodel.fit(X_train, y_train)
time_1 = time.time()
pred_train = rfmodel.predict(X_train)
time_2 = time.time()
pred_test = rfmodel.predict(X_test)
time_3 = time.time()
print('Time to train:', time_1-time_0, 'seconds')
print('Time to predict train:', time_2-time_1, 'seconds')
print('Time to predict test:', time_3-time_2, 'seconds')
print('Total time:', time_3-time_0, 'seconds')
print('Train accuracy:',accuracy_score(y_train, pred_train))
print('\nConfusion matrix (suppTrump, prediction):\n',confusion_matrix(y_test, pred_test))
print('Test accuracy:',accuracy_score(y_test, pred_test))

Time to train: 6.531135082244873 seconds
Time to predict train: 1.6178903579711914 seconds
Time to predict test: 0.634434700012207 seconds
Total time: 8.783460140228271 seconds
Train accuracy: 0.9736356587074521

Confusion matrix (suppTrump, prediction):
 [[17565  1033]
 [ 1208 12251]]
Test accuracy: 0.9300932713603893


## Estimating likelihood of lying 

In [25]:
X_context_vote = X_context[(X_context['suppTrump']==1) & (X_context['vote16_nan']==0)].drop(columns={'suppTrump','vote16_donaldtrump','vote16_nan'})
y_vote = (X_context[(X_context['suppTrump']==1) & (X_context['vote16_nan']==0)]['vote16_donaldtrump'])*-1+1

X_train_vote, X_test_vote, y_train_vote, y_test_vote = train_test_split(X_context_vote, y_vote, test_size=0.3, random_state=1)

print(X_train_vote.shape)
print(y_train_vote.shape)
print(X_test_vote.shape)
print(y_test_vote.shape)

(8108, 53)
(8108,)
(3476, 53)
(3476,)


In [26]:
X_train_vote_intercept = X_train_vote.copy(deep = True)

X_train_vote_intercept['intercept'] = 1
logmodel_vars_vote = sm.Logit(y_train_vote,X_train_vote_intercept)
result = logmodel_vars_vote.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.273546
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:     vote16_donaldtrump   No. Observations:                 8108
Model:                          Logit   Df Residuals:                     8054
Method:                           MLE   Df Model:                           53
Date:                Fri, 20 Mar 2020   Pseudo R-squ.:                  0.2250
Time:                        13:17:13   Log-Likelihood:                -2217.9
converged:                       True   LL-Null:                       -2861.7
Covariance Type:            nonrobust   LLR p-value:                1.452e-234
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
mode                           -0.1458      0.170     -0.857      0.391   

In [27]:
time_0 = time.time()
logmodel_vote = LogisticRegression(max_iter=300)
logmodel_vote.fit(X_train_vote, y_train_vote)
time_1 = time.time()
pred_train_vote = logmodel_vote.predict(X_train_vote)
time_2 = time.time()
pred_test_vote = logmodel_vote.predict(X_test_vote)
time_3 = time.time()
print('Time to train:', time_1-time_0, 'seconds')
print('Time to predict train:', time_2-time_1, 'seconds')
print('Time to predict test:', time_3-time_2, 'seconds')
print('Total time:', time_3-time_0, 'seconds')
print('Train accuracy:',accuracy_score(y_train_vote, pred_train_vote))
print('\nConfusion matrix (suppTrump, prediction):\n',confusion_matrix(y_test_vote, pred_test_vote))
print('Test accuracy:',accuracy_score(y_test_vote, pred_test_vote))

Time to train: 0.38544297218322754 seconds
Time to predict train: 0.0032334327697753906 seconds
Time to predict test: 0.002293109893798828 seconds
Total time: 0.39096951484680176 seconds
Train accuracy: 0.899605328071041

Confusion matrix (suppTrump, prediction):
 [[3029   57]
 [ 306   84]]
Test accuracy: 0.8955696202531646


In [28]:
time_0 = time.time()
rfmodel_vote = RandomForestClassifier(n_estimators=100, max_depth=15)
rfmodel_vote.fit(X_train_vote, y_train_vote)
time_1 = time.time()
pred_train_vote = rfmodel_vote.predict(X_train_vote)
time_2 = time.time()
pred_test_vote = rfmodel_vote.predict(X_test_vote)
time_3 = time.time()
print('Time to train:', time_1-time_0, 'seconds')
print('Time to predict train:', time_2-time_1, 'seconds')
print('Time to predict test:', time_3-time_2, 'seconds')
print('Total time:', time_3-time_0, 'seconds')
print('Train accuracy:',accuracy_score(y_train_vote, pred_train_vote))
print('\nConfusion matrix (suppTrump, prediction):\n',confusion_matrix(y_test_vote, pred_test_vote))
print('Test accuracy:',accuracy_score(y_test_vote, pred_test_vote))

Time to train: 0.7584538459777832 seconds
Time to predict train: 0.14068126678466797 seconds
Time to predict test: 0.07213139533996582 seconds
Total time: 0.971266508102417 seconds
Train accuracy: 0.972496299950666

Confusion matrix (suppTrump, prediction):
 [[3062   24]
 [ 342   48]]
Test accuracy: 0.8947065592635213


# Plotting

## Prep

In [29]:
full_X = pd.concat([X_train, X_test])

df_trump['logit_pred'] = logmodel.predict(full_X)
df_trump['rfmodel_pred'] = rfmodel.predict(full_X)

print(df_trump.logit_pred.value_counts())
print(df_trump.rfmodel_pred.value_counts())

0.0    62654
1.0    44201
Name: logit_pred, dtype: int64
0.0    62578
1.0    44277
Name: rfmodel_pred, dtype: int64


In [0]:
df_k = df_trump.copy(deep=True)

In [0]:
def LogitSupp(row):
  if row['logit_pred'] == 1:
    return 'Logit_Support'
  else:
    return 'Logit_Oppose'
df_k['LogitSupp'] = df_k.apply(LogitSupp, axis=1)

In [32]:
print(df_k.LogitSupp.value_counts(normalize=True).sort_index()*100)
print(df_k.LogitSupp.value_counts().sort_index())

Logit_Oppose     58.634598
Logit_Support    41.365402
Name: LogitSupp, dtype: float64
Logit_Oppose     62654
Logit_Support    44201
Name: LogitSupp, dtype: int64


In [0]:
def RFSupp(row):
  if row['rfmodel_pred'] == 1:
    return 'RF_Support'
  else:
    return 'RF_Oppose'
df_k['RFSupp'] = df_k.apply(RFSupp, axis=1)

In [34]:
print(df_k.RFSupp.value_counts(normalize=True).sort_index()*100)
print(df_k.RFSupp.value_counts().sort_index())

RF_Oppose     58.563474
RF_Support    41.436526
Name: RFSupp, dtype: float64
RF_Oppose     62578
RF_Support    44277
Name: RFSupp, dtype: int64


In [0]:
def TrumpSupp(row):
  if row['suppTrump'] == 1:
    return 'Support'
  else:
    return 'Oppose'
df_k['TrumpSupp'] = df_k.apply(TrumpSupp, axis=1)

In [36]:
print(df_k.TrumpSupp.value_counts(normalize=True).sort_index()*100)
print(df_k.TrumpSupp.value_counts().sort_index())

Oppose     58.143278
Support    41.856722
Name: TrumpSupp, dtype: float64
Oppose     62129
Support    44726
Name: TrumpSupp, dtype: int64


In [37]:
trump = pd.crosstab(index=[df_k.TrumpSupp]
            , columns=[df_k.TrumpSupp])
trump = trump.reset_index()

trump_logit = pd.crosstab(index=[df_k.LogitSupp]
            , columns=[df_k.LogitSupp])
trump_logit = trump_logit.reset_index()

trump_rf = pd.crosstab(index=[df_k.RFSupp]
            , columns=[df_k.RFSupp])
trump_rf = trump_rf.reset_index()

print(trump)
print(trump_logit)
print(trump_rf)

TrumpSupp TrumpSupp  Oppose  Support
0            Oppose   62129        0
1           Support       0    44726
LogitSupp      LogitSupp  Logit_Oppose  Logit_Support
0           Logit_Oppose         62654              0
1          Logit_Support             0          44201
RFSupp      RFSupp  RF_Oppose  RF_Support
0        RF_Oppose      62578           0
1       RF_Support          0       44277


In [38]:
df_k.age_18_24.value_counts()

0    99550
1     7305
Name: age_18_24, dtype: int64

In [0]:
def AgeGroup(row):
  if row['age_18_24'] == 1:
    return 'age_18_29'
  elif row['age_25_29'] == 1:
    return 'age_18_29'
  elif row['age_30_39'] == 1:
    return 'age_30_39'
  elif row['age_40_49'] == 1:
    return 'age_40_49'
  elif row['age_50_64'] == 1:
    return 'age_50_64'
  elif row['age_65+'] == 1:
    return 'age_65+'
  else:
    return 'other'
df_k['AgeGroup'] = df_k.apply(AgeGroup, axis=1)

In [40]:
print(df_k.AgeGroup.value_counts(normalize=True).sort_index()*100)
print(df_k.AgeGroup.value_counts().sort_index())

age_18_29    14.700295
age_30_39    17.672547
age_40_49    14.453231
age_50_64    29.224650
age_65+      23.949277
Name: AgeGroup, dtype: float64
age_18_29    15708
age_30_39    18884
age_40_49    15444
age_50_64    31228
age_65+      25591
Name: AgeGroup, dtype: int64


In [41]:
age_trump = pd.crosstab(index=[df_k.AgeGroup]
            , columns=[df_k.TrumpSupp])
age_trump = age_trump.reset_index()

age_trump_logit = pd.crosstab(index=[df_k.AgeGroup]
            , columns=[df_k.LogitSupp])
age_trump_logit = age_trump_logit.reset_index()

age_trump_rf = pd.crosstab(index=[df_k.AgeGroup]
            , columns=[df_k.RFSupp])
age_trump_rf = age_trump_rf.reset_index()

print(age_trump)
print(age_trump_logit)
print(age_trump_rf)

TrumpSupp   AgeGroup  Oppose  Support
0          age_18_29   10575     5133
1          age_30_39   11816     7068
2          age_40_49    8800     6644
3          age_50_64   17070    14158
4            age_65+   13868    11723
LogitSupp   AgeGroup  Logit_Oppose  Logit_Support
0          age_18_29          9232           6476
1          age_30_39         11049           7835
2          age_40_49          8979           6465
3          age_50_64         18313          12915
4            age_65+         15081          10510
RFSupp   AgeGroup  RF_Oppose  RF_Support
0       age_18_29       9206        6502
1       age_30_39      11075        7809
2       age_40_49       8964        6480
3       age_50_64      18243       12985
4         age_65+      15090       10501


In [0]:
def Gender(row):
  if row['sex_m'] == 1:
    return 'Male'
  else:
    return 'Female'
df_k['Gender'] = df_k.apply(Gender, axis=1)

In [43]:
print(df_k.Gender.value_counts(normalize=True).sort_index()*100)
print(df_k.Gender.value_counts().sort_index())

Female    61.002293
Male      38.997707
Name: Gender, dtype: float64
Female    65184
Male      41671
Name: Gender, dtype: int64


In [44]:
gender_trump = pd.crosstab(index=[df_k.Gender]
            , columns=[df_k.TrumpSupp])
gender_trump = gender_trump.reset_index()

gender_trump_logit = pd.crosstab(index=[df_k.Gender]
            , columns=[df_k.LogitSupp])
gender_trump_logit = gender_trump_logit.reset_index()

gender_trump_rf = pd.crosstab(index=[df_k.Gender]
            , columns=[df_k.RFSupp])
gender_trump_rf = gender_trump_rf.reset_index()

print(gender_trump)
print(gender_trump_logit)
print(gender_trump_rf)

TrumpSupp  Gender  Oppose  Support
0          Female   40301    24883
1            Male   21828    19843
LogitSupp  Gender  Logit_Oppose  Logit_Support
0          Female         38306          26878
1            Male         24348          17323
RFSupp  Gender  RF_Oppose  RF_Support
0       Female      38265       26919
1         Male      24313       17358


In [0]:
def PartyGroup(row):
  if row['party'] == 0:
    return 'Democratic'
  elif row['party'] == 1:
    return 'Democratic_Lean'
  elif row['party'] == 2:
    return 'Independent'
  elif row['party'] == 3:
    return 'Republican_Lean'
  elif row['party'] == 4:
    return 'Republican'
  else:
    return 'other'
df_k['PartyGroup'] = df_k.apply(PartyGroup, axis=1)


In [46]:
print(df_k.PartyGroup.value_counts(normalize=True).sort_index()*100)
print(df_k.PartyGroup.value_counts().sort_index())

Democratic         39.403865
Democratic_Lean    10.712648
Independent        11.508118
Republican         29.871321
Republican_Lean     8.504048
Name: PartyGroup, dtype: float64
Democratic         42105
Democratic_Lean    11447
Independent        12297
Republican         31919
Republican_Lean     9087
Name: PartyGroup, dtype: int64


In [47]:
party_trump = pd.crosstab(index=[df_k.PartyGroup]
            , columns=[df_k.TrumpSupp])
party_trump = party_trump.reset_index()

party_trump_logit = pd.crosstab(index=[df_k.PartyGroup]
            , columns=[df_k.LogitSupp])
party_trump_logit = party_trump_logit.reset_index()

party_trump_rf = pd.crosstab(index=[df_k.PartyGroup]
            , columns=[df_k.RFSupp])
party_trump_rf = party_trump_rf.reset_index()

print(party_trump)
print(party_trump_logit)
print(party_trump_rf)

TrumpSupp       PartyGroup  Oppose  Support
0               Democratic   38385     3720
1          Democratic_Lean   10740      707
2              Independent    7309     4988
3               Republican    4041    27878
4          Republican_Lean    1654     7433
LogitSupp       PartyGroup  Logit_Oppose  Logit_Support
0               Democratic         24707          17398
1          Democratic_Lean          6655           4792
2              Independent          7204           5093
3               Republican         18621          13298
4          Republican_Lean          5467           3620
RFSupp       PartyGroup  RF_Oppose  RF_Support
0            Democratic      24654       17451
1       Democratic_Lean       6680        4767
2           Independent       7176        5121
3            Republican      18615       13304
4       Republican_Lean       5453        3634


In [0]:
def EthnicGroup(row):
  if row['raceth_hsp'] == 1:
    return 'latinx'
  elif row['raceth_blk'] == 1:
    return 'afam'
  elif row['raceth_asn'] == 1:
    return 'asian'
  elif row['raceth_hipac'] == 1:
    return 'asian'
  elif row['raceth_wht'] == 1:
    return 'white'
  elif row['raceth_ntvam'] == 1:
    return 'native'
  else:
    return 'other'
df_k['eth'] = df_k.apply(EthnicGroup, axis=1)

In [49]:
print(df_k.eth.value_counts(normalize=True).sort_index()*100)
print(df_k.eth.value_counts().sort_index())

afam       8.918628
asian      2.632539
latinx     7.836788
native     0.440784
other      4.309578
white     75.861682
Name: eth, dtype: float64
afam       9530
asian      2813
latinx     8374
native      471
other      4605
white     81062
Name: eth, dtype: int64


In [50]:
eth_trump = pd.crosstab(index=[df_k.eth]
            , columns=[df_k.TrumpSupp])
eth_trump = eth_trump.reset_index()

eth_trump_logit = pd.crosstab(index=[df_k.eth]
            , columns=[df_k.LogitSupp])
eth_trump_logit = eth_trump_logit.reset_index()

eth_trump_rf = pd.crosstab(index=[df_k.eth]
            , columns=[df_k.RFSupp])
eth_trump_rf = eth_trump_rf.reset_index()

print(eth_trump)
print(eth_trump_logit)
print(eth_trump_rf)

TrumpSupp     eth  Oppose  Support
0            afam    8307     1223
1           asian    1927      886
2          latinx    5606     2768
3          native     271      200
4           other    2835     1770
5           white   43183    37879
LogitSupp     eth  Logit_Oppose  Logit_Support
0            afam          5598           3932
1           asian          1649           1164
2          latinx          4882           3492
3          native           288            183
4           other          2736           1869
5           white         47501          33561
RFSupp     eth  RF_Oppose  RF_Support
0         afam       5583        3947
1        asian       1641        1172
2       latinx       4894        3480
3       native        287         184
4        other       2747        1858
5        white      47426       33636


## Plots

In [0]:
chart_studio.tools.set_credentials_file(username='kielbrunner', api_key='skeo2zBonhX5ayjYv0AC')

In [52]:
fig = go.Figure(data=[
    go.Bar(name='Oppose'
    , x=trump.TrumpSupp
    , y=trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=trump_logit.LogitSupp
    , y=trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=trump_rf.RFSupp
    , y=trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=trump.TrumpSupp
    , y=trump.Support),
    go.Bar(name='Logit_Support'
    , x=trump_logit.LogitSupp
    , y=trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=trump_rf.RFSupp
    , y=trump_rf.RF_Support),
])
#fig.update_layout(barmode='group')
fig.update_layout(title_text="Trump Support<br>Both Models Appear to Match the Survey Questions")

fig.show()

In [53]:
fig_age = go.Figure(data=[
    go.Bar(name='Oppose'
    , x=age_trump.AgeGroup
    , y=age_trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=age_trump_logit.AgeGroup
    , y=age_trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=age_trump_rf.AgeGroup
    , y=age_trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=age_trump.AgeGroup
    , y=age_trump.Support),
    go.Bar(name='Logit_Support'
    , x=age_trump_logit.AgeGroup
    , y=age_trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=age_trump_rf.AgeGroup
    , y=age_trump_rf.RF_Support),
])
fig_age.update_layout(barmode='group')
fig_age.update_layout(title_text="Trump Support by Age<br>Both Models Tracked Well with the Survey Questions but Understated Support/Opposition")

fig_age.show()

In [54]:
fig_party = go.Figure(data=[
    go.Bar(name='Oppose'
    , x=party_trump.PartyGroup
    , y=party_trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=party_trump_logit.PartyGroup
    , y=party_trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=party_trump_rf.PartyGroup
    , y=party_trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=party_trump.PartyGroup
    , y=party_trump.Support),
    go.Bar(name='Logit_Support'
    , x=party_trump_logit.PartyGroup
    , y=party_trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=party_trump_rf.PartyGroup
    , y=party_trump_rf.RF_Support),
])

fig_party.update_layout(barmode='group')
fig_party.update_layout(title_text="Trump Support and Models by Party: <br>Clear divide along party lines and Independents Slightly Opposed. Both Models Trend Well but Fall Short of Survey Responses")

fig_party.show()

In [55]:
fig_sex = go.Figure(data=[

    go.Bar(name='Oppose'
    , x=gender_trump.Gender
    , y=gender_trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=gender_trump_logit.Gender
    , y=gender_trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=gender_trump_rf.Gender
    , y=gender_trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=gender_trump.Gender
    , y=gender_trump.Support),
    go.Bar(name='Logit_Support'
    , x=gender_trump_logit.Gender
    , y=gender_trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=gender_trump_rf.Gender
    , y=gender_trump_rf.RF_Support),
])
fig_sex.update_layout(barmode='group')
fig_sex.update_layout(title_text="Trump Support by Sex<br>Both Models Made Accurate Predictions Across the Gender Divide, Although Support/Opposition was Mildly Overstated")

fig_sex.show()

In [56]:
fig_eth = go.Figure(data=[

    go.Bar(name='Oppose'
    , x=eth_trump.eth
    , y=eth_trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=eth_trump_logit.eth
    , y=eth_trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=eth_trump_rf.eth
    , y=eth_trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=eth_trump.eth
    , y=eth_trump.Support),
    go.Bar(name='Logit_Support'
    , x=eth_trump_logit.eth
    , y=eth_trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=eth_trump_rf.eth
    , y=eth_trump_rf.RF_Support),
])

fig_eth.update_layout(barmode='group')
fig_eth.update_layout(title_text="Trump Support by Ethnicity<br>The Model Trends in the Direction Among White Voters, While Other Ethnicities are Tough to Discern")

fig_eth.show()

In [57]:
fig_eth_nw = go.Figure(data=[

    go.Bar(name='Oppose'
    , x=eth_trump.eth[eth_trump.eth != 'white']
    , y=eth_trump.Oppose),
    go.Bar(name='Logit_Oppose'
    , x=eth_trump_logit.eth[eth_trump.eth != 'white']
    , y=eth_trump_logit.Logit_Oppose),
     go.Bar(name='RF_Oppose'
    , x=eth_trump_rf.eth[eth_trump.eth != 'white']
    , y=eth_trump_rf.RF_Oppose),   
    go.Bar(name='Support'
    , x=eth_trump.eth[eth_trump.eth != 'white']
    , y=eth_trump.Support),
    go.Bar(name='Logit_Support'
    , x=eth_trump_logit.eth[eth_trump.eth != 'white']
    , y=eth_trump_logit.Logit_Support),
    go.Bar(name='RF_Support'
    , x=eth_trump_rf.eth[eth_trump.eth != 'white']
    , y=eth_trump_rf.RF_Support),
])

fig_eth_nw.update_layout(barmode='group')
fig_eth_nw.update_layout(title_text="Trump Support by Non-white Ethnicity<br>Among Non-white Ethnicities, the Models Follow the Survey Questions but Understate Support/Opposition")

fig_eth_nw.show()

In [58]:
print(py.plot(fig, filename = 'Trump Model', auto_open=True))
print(py.plot(fig_sex, filename = 'Trump Gender', auto_open=True))
print(py.plot(fig_party, filename = 'Trump Party', auto_open=True))
print(py.plot(fig_eth, filename = 'Trump Ethnicity', auto_open=True))
print(py.plot(fig_eth_nw, filename = 'Trump Ethnicity Non-White', auto_open=True))
print(py.plot(fig_age, filename = 'Trump Age', auto_open=True))

https://plot.ly/~kielbrunner/55/
https://plot.ly/~kielbrunner/61/
https://plot.ly/~kielbrunner/59/
https://plot.ly/~kielbrunner/133/
https://plot.ly/~kielbrunner/137/
https://plot.ly/~kielbrunner/69/


# User Interface

## Prep

In [59]:
# rebase dataframe
df_form = full_data.copy(deep=True).replace({'(99) DON’T KNOW/SKIPPED/REFUSED (VOL)':np.NaN}).replace({'(99) DON\'T KNOW/SKIPPED/REFUSED (VOL)':np.NaN})

df_form = df_form.drop(columns={'su_id','p_state','race1_vote','race1_party','race2_vote','race2_party','race3_vote','race3_party','race4_vote','race4_party','race5_vote','race5_party'})
df_form = df_form.drop(columns={'pollclose_state_weight','pollclose_national_weight','finalvote_state_weight','finalvote_national_weight','statenum','lvb'})
df_form = df_form.drop(columns={'race1_raceid','race1_officeid','race1_racetypeid','race2_raceid','race2_officeid','race2_racetypeid','race3_raceid',\
                      'race3_officeid','race3_racetypeid','race4_raceid','race4_officeid','race4_racetypeid','race5_raceid','race5_officeid',\
                      'race5_racetypeid','ballot1_vote','ballot1_party','ballot1_raceid','ballot1_officeid','ballot1_racetypeid','senvote',\
                      'senspvote','govvote','mainechoice2','favsenind','favgovind'})
df_form = df_form.drop(columns={'favfeinstein','favdeleon','favssenspecialdem','favssenspecialrep','favespy','favhydesmith','favmcdaniel'})
df_form = df_form.drop(columns={'married','gunowner','union','vet'})
df_form = df_form.drop(columns={'govjobkasich4','govjobsnyder4','favsessions','favjones','favpalin','favflake','favcotton','favboozman','favemanuel','favpence','favfranken','favinslee','sportsbetting'})
df_form = df_form.drop(columns={'akdividends','ilbudget','prostitution','incometax','murkowskisc','collinssc','gastax','harrispres',\
                          'murphypres','bidenpres','mapreswarren','maprespatrick','nypresbloomberg','nypresgillibrand','klobucharpres','merkleypres','sanderspres','unfairflgillum',\
                          'unfairfldesantis','unfairnjmenendez','unfairnjhugin','comarijuana','redtide','firstblackgov','allowedvote','kstax',\
                          'mshydesmithvsespy','msmcdanielvsespy','secondchancelaw','govworker','coalhh','tradedemvsrep'})
df_form = df_form.drop(columns={'honestmenendez','honesthugin','taylor','orourketooliberal','cruzfocus','romneytrump','wnv','breakc','cuban','firsttime','formflag'})
df_form = df_form.drop(columns={'qpvvote','q2016vote','q2020vote','presapp4','trumpfactor3','favtrump'})
df_form = df_form.dropna(thresh=len(df_form)-11000, axis=1)
df_form = df_form.replace({'(88) REMOVED FOR DISCLOSURE RISK':np.NaN})
print(df_form.shape)
print(df_form.head(10).to_string())

(138929, 28)
        mode                       lva                         lv       likelyvoter            timevote                track                    issues            getahead  tradenationalecon tradelocalecon                    healthlaw                                       immdeport   russia                                           qpvvote3             breaka             breakb        sex      age65                  raceth                        raceth5                            educ                income           party                partyfull                       ideo                relig                          relig4       sizeplace
0    (2) Web  (1) Extremely interested         (12) Already voted  (1) Likely voter  (2) Early/Absentee  (2) Wrong direction       (7) The environment  (2) Holding steady           (2) Hurt       (2) Hurt           (4) Expand the law  (1) Offered a chance to apply for legal status  (1) Yes                               (4) I'm sure I vot

In [0]:
question = ['How are you completing this survey?',\
            #'In which state are you registered to vote?',\
            'How interested are you in the election taking place on November 6th, 2018?',\
            'On a scale from zero to 10, where 10 means you\'re certain you will vote and zero means there is no chance you will vote, please indicate how likely it is that you will vote in the 2018 midterm election.',\
            '',\
            'In 2018, did you vote at a polling place on Election Day, or did you vote by mail or vote in person at some point before Election Day?',\
            'Generally speaking, would you say things in this country are heading in the…',\
            'Which one of the following would you say is the most important issue facing the country?',\
            'Which best describes your family\'s financial situation? Are you…',\
            'Do you think the Trump administration\'s trade policies will help or hurt the national economy, or won\'t they make much difference?',\
            'Do you think the Trump administration\'s trade policies will help or hurt the economy in your area, or won\'t they make much difference?',\
            'Which of the following comes closest to what you would like to see lawmakers do with the Affordable Care Act, also known as Obamacare?',\
            'Which of the following comes closer to your opinion? Should most immigrants who are living in the United States illegally be…',\
            'Do you think the Trump campaign coordinated with Russia during the 2016 presidential election, or not?',\
            'Regarding the 2014 election for Congress, which one of the following statements best describes you?',\
            'Do you think the way Democrats talk about politics these days is leading to an increase in acts of violence, or don\'t you think so?',\
            'Do you think the way Republicans talk about politics these days is leading to an increase in acts of violence, or don\'t you think so?',\
            'Are you male or female?',\
            'What is your age?',\
            'What is your racial or ethnic heritage?',\
            '',\
            'Which best describes your level of education?',\
            'In which of these categories is your total yearly household income, that is, of everyone living in your family household? Was your total household income in 2017…',\
            'Generally speaking, do you consider yourself a Democrat, a Republican, or don\'t you consider yourself either?',\
            'Would you say you\'re closer to the Democrats, to the Republicans, or don\'t you lean toward either?',\
            'On most political matters, do you consider yourself…',\
            'What is your religion, if any?',\
            '',\
            'Which of the following best describes the area where you live?']

In [61]:
variable = []
attributes = []
for i in range(df_form.shape[1]):
  var = df_form.columns[i]
  attr = df_form[var].dropna().sort_values().unique()
  if var == 'ideo':
    variable.append('ideol')
  else:
    variable.append(var)
  attributes.append(attr)
df_form_ref = pd.DataFrame(list(zip(variable,question,attributes)),columns=['variable','question','attributes'])
print(df_form_ref.to_string())

             variable                                           question                                         attributes
0                mode                How are you completing this survey?                               [(1) Phone, (2) Web]
1                 lva  How interested are you in the election taking ...  [(1) Extremely interested, (2) Very interested...
2                  lv  On a scale from zero to 10, where 10 means you...  [(0) 0-Certain will not vote, (1) 1, (10) 10-C...
3         likelyvoter                                                                      [(1) Likely voter, (2) Nonvoter]
4            timevote  In 2018, did you vote at a polling place on El...             [(1) Election Day, (2) Early/Absentee]
5               track  Generally speaking, would you say things in th...         [(1) Right direction, (2) Wrong direction]
6              issues  Which one of the following would you say is th...  [(1) The economy and jobs, (2) Health care, (3...
7       

In [62]:
print('#@title Propensity Estimate of Trump Support')
inputs = 'ui_input = ['
q_n = 1
for i in range(df_form_ref.shape[0]):
  q = df_form_ref['question'][i]
  v = df_form_ref['variable'][i]
  a = df_form_ref['attributes'][i]
  a_str = '['
  for j in range(len(a)):
    a_str = str(a_str + '"' + a[j] + '"')
    if j < len(a)-1:
      a_str = str(a_str + ", ")
    else:
      a_str = str(a_str + "]")
  if v == 'lv':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print('lv = "(0) 0-Certain will not vote" #@param ["(0) 0-Certain will not vote", "(1) 1", "(2) 2", "(3) 3", "(4) 4", "(5) 5", "(6) 6", "(7) 7", "(8) 8", "(9) 9", "(10) 10-Certain will vote", "(12) Already voted"]')
    print()
  elif v == 'likelyvoter':
    print('if lv in ("(10) 10-Certain will vote", "(12) Already voted", "(8) 8", "(9) 9"):')
    print('  likelyvoter = "(1) Likely voter"')
    print('else:')
    print('  likelyvoter = "(2) Nonvoter"')
    print()
  elif v == 'issues':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('issues_economyandjobs = 0')
    print('issues_healthcare = 0')
    print('issues_immigration = 0')
    print('issues_taxes = 0')
    print('issues_abortion = 0')
    print('issues_gunpolicy = 0')
    print('issues_environment = 0')
    print('issues_foreignpolicy = 0')
    print('if issues == "(1) The economy and jobs":')
    print('  issues_economyandjobs = 1')
    print('elif issues == "(2) Health care":')
    print('  issues_healthcare = 1')
    print('elif issues == "(3) Immigration":')
    print('  issues_immigration = 1')
    print('elif issues == "(4) Taxes":')
    print('  issues_taxes = 1')
    print('elif issues == "(5) Abortion":')
    print('  issues_abortion = 1')
    print('elif issues == "(6) Gun policy":')
    print('  issues_gunpolicy = 1')
    print('elif issues == "(7) The environment":')
    print('  issues_environment = 1')
    print('elif issues == "(8) Foreign policy":')
    print('  issues_foreignpolicy = 1')
    print()
  elif v == 'age65':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('age_18_24 = 0')
    print('age_25_29 = 0')
    print('age_30_39 = 0')
    print('age_40_49 = 0')
    print('age_50_64 = 0')
    print('if age65 == "(1) 18-24":')
    print('  age_18_24 = 1')
    print('elif age65 == "(2) 25-29":')
    print('  age_25_29 = 1')
    print('elif age65 == "(3) 30-39":')
    print('  age_30_39 = 1')
    print('elif age65 == "(4) 40-49":')
    print('  age_40_49 = 1')
    print('elif age65 == "(5) 50-64":')
    print('  age_50_64 = 1')
    print()
  elif v == 'raceth':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('raceth_wht = 0')
    print('raceth_blk = 0')
    print('raceth_hsp = 0')
    print('raceth_asn = 0')
    print('raceth_ntvam = 0')
    print('raceth_hipac = 0')
    print('if raceth == "(1) White":')
    print('  raceth_wht = 1')
    print('elif raceth == "(2) African American or Black":')
    print('  raceth_blk = 1')
    print('elif raceth == "(3) Latino or Hispanic":')
    print('  raceth_hsp = 1')
    print('elif raceth == "(4) Asian":')
    print('  raceth_asn = 1')
    print('elif raceth == "(5) American Indian, Native American or Alaska Native":')
    print('  raceth_ntvam = 1')
    print('elif raceth == "(6) Native Hawaiian or Other Pacific Islander":')
    print('  raceth_hipac = 1')
    print()
  elif v == 'educ':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('edu_highschoolorless = 0')
    print('edu_somecollege_assocdegree = 0')
    print('edu_collegegraduate = 0')
    print('if educ == "(1) High school or less":')
    print('  edu_highschoolorless = 1')
    print('elif educ == "(2) Some college/assoc. degree":')
    print('  edu_somecollege_assocdegree = 1')
    print('elif educ == "(3) College graduate":')
    print('  edu_collegegraduate = 1')
    print()
  elif v == 'income':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('income_under25000 = 0')
    print('income_25000_49999 = 0')
    print('income_50000_74999 = 0')
    print('income_75000_99999 = 0')
    print('if income == "(1) Under $25,000":')
    print('  income_under25000 = 1')
    print('elif income == "(2) $25,000-$49,999":')
    print('  income_25000_49999 = 1')
    print('elif income == "(3) $50,000-$74,999":')
    print('  income_50000_74999 = 1')
    print('elif income == "(4) $75,000-$99,999":')
    print('  income_75000_99999 = 1')
    print()
  elif v == 'relig':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('relig_prot_chrst = 0')
    print('relig_catholic = 0')
    print('relig_mormon = 0')
    print('relig_jewish = 0')
    print('relig_muslim = 0')
    print('relig_none = 0')
    print('if relig == "(1) Protestant" or relig == "(4) Other Christian":')
    print('  relig_prot_chrst = 1')
    print('elif relig == "(2) Catholic":')
    print('  relig_catholic = 1')
    print('elif relig == "(3) Mormon":')
    print('  relig_mormon = 1')
    print('elif relig == "(5) Jewish":')
    print('  relig_jewish = 1')
    print('elif relig == "(6) Muslim":')
    print('  relig_muslim = 1')
    print('elif relig == "(8) None":')
    print('  relig_none = 1')
    print()
  elif v == 'sizeplace':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print('sizeplace_urban = 0')
    print('sizeplace_suburban = 0')
    print('sizeplace_smalltown = 0')
    print('if sizeplace == "(1) Urban":')
    print('  sizeplace_urban = 1')
    print('elif sizeplace == "(2) Suburban":')
    print('  sizeplace_suburban = 1')
    print('elif sizeplace == "(3) Small town":')
    print('  sizeplace_smalltown = 1')
    print()
  elif q != '':
    print(str('#@markdown ' + str(q_n) + '. ' + q))
    q_n += 1
    print(str(v + ' = "' + a[0] + '" #@param ' + a_str))
    print()
  else:
    print(str(v + ' = None'))
    print()
  inputs = str(inputs + v)
  if i < df_form_ref.shape[0]-1:
    inputs = str(inputs + ',')
  else:
    inputs = str(inputs + ']')

print(str('print(\'Inputs saved!\\n\')'))
print('ui_input = pd.DataFrame([[mode,lva,lv,likelyvoter,timevote,track,issues_economyandjobs,issues_healthcare,issues_immigration,issues_taxes,issues_abortion,issues_gunpolicy,issues_environment,issues_foreignpolicy,getahead,tradenationalecon,tradelocalecon,healthlaw,immdeport,russia,qpvvote3,breaka,breakb,sex,age_18_24,age_25_29,age_30_39,age_40_49,age_50_64,raceth_wht,raceth_blk,raceth_hsp,raceth_asn,raceth_ntvam,raceth_hipac,edu_highschoolorless,edu_somecollege_assocdegree,edu_collegegraduate,income_under25000,income_25000_49999,income_50000_74999,income_75000_99999,party,partyfull,ideol,relig_prot_chrst,relig_catholic,relig_mormon,relig_jewish,relig_muslim,relig_none,sizeplace_urban,sizeplace_suburban,sizeplace_smalltown]],columns=(\'mode\',\'lva\',\'lv\',\'likelyvoter\',\'timevote\',\'track\',\'issues_economyandjobs\',\'issues_healthcare\',\'issues_immigration\',\'issues_taxes\',\'issues_abortion\',\'issues_gunpolicy\',\'issues_environment\',\'issues_foreignpolicy\',\'getahead\',\'tradenationalecon\',\'tradelocalecon\',\'healthlaw\',\'immdeport\',\'russia\',\'qpvvote3\',\'breaka\',\'breakb\',\'sex\',\'age_18_24\',\'age_25_29\',\'age_30_39\',\'age_40_49\',\'age_50_64\',\'raceth_wht\',\'raceth_blk\',\'raceth_hsp\',\'raceth_asn\',\'raceth_ntvam\',\'raceth_hipac\',\'edu_highschoolorless\',\'edu_somecollege_assocdegree\',\'edu_collegegraduate\',\'income_under25000\',\'income_25000_49999\',\'income_50000_74999\',\'income_75000_99999\',\'party\',\'partyfull\',\'ideo\',\'relig_prot_chrst\',\'relig_catholic\',\'relig_mormon\',\'relig_jewish\',\'relig_muslim\',\'relig_none\',\'sizeplace_urban\',\'sizeplace_suburban\',\'sizeplace_smalltown\'))')
print('ui_input = call_to_binary(ui_input,\'mode\',val_1=2)')
print('ui_input = call_to_ranked_reverse(ui_input,\'lva\',\'electinterest\',5)')
print('ui_input = call_to_prob(ui_input,\'lv\',\'P_vote\')')
print('ui_input = call_to_binary(ui_input,\'likelyvoter\')')
print('ui_input = call_to_binary(ui_input,\'timevote\')')
print('ui_input = call_to_binary(ui_input,\'track\')')
print('ui_input = call_to_ranked_reverse(ui_input,\'getahead\',\'famfinancial\',3)')
print('ui_input = call_to_3_choice_flip(ui_input,\'tradenationalecon\',\'trumpnatecon\')')
print('ui_input = call_to_3_choice_flip(ui_input,\'tradelocalecon\',\'trumplocalecon\')')
print('ui_input = call_to_ranked_reverse(ui_input,\'healthlaw\',\'supACA\',4)')
print('ui_input = call_to_binary(ui_input,\'immdeport\',\'immpol\')')
print('ui_input = call_to_binary(ui_input,\'russia\',\'russiacollude16\')')
print('ui_input = call_to_binary(ui_input,\'qpvvote3\',\'voted14\',4)')
print('ui_input = call_to_binary(ui_input,\'breaka\',\'violencespeechD\')')
print('ui_input = call_to_binary(ui_input,\'breakb\',\'violencespeechR\')')
print('ui_input = call_to_binary(ui_input,\'sex\',\'sex_m\')')
print('ui_input = call_combo_party(ui_input)')
print('ui_input = call_ideo(ui_input)')
print('pred = logmodel.predict_proba(ui_input)')
print('print("P(suppTrump=0) P(supTrump=1)")')
print('print(pred)')


#@title Propensity Estimate of Trump Support
#@markdown 1. How are you completing this survey?
mode = "(1) Phone" #@param ["(1) Phone", "(2) Web"]

#@markdown 2. How interested are you in the election taking place on November 6th, 2018?
lva = "(1) Extremely interested" #@param ["(1) Extremely interested", "(2) Very interested", "(3) Somewhat interested", "(4) Only a little interested", "(5) Not at all interested"]

#@markdown 3. On a scale from zero to 10, where 10 means you're certain you will vote and zero means there is no chance you will vote, please indicate how likely it is that you will vote in the 2018 midterm election.
lv = "(0) 0-Certain will not vote" #@param ["(0) 0-Certain will not vote", "(1) 1", "(2) 2", "(3) 3", "(4) 4", "(5) 5", "(6) 6", "(7) 7", "(8) 8", "(9) 9", "(10) 10-Certain will vote", "(12) Already voted"]

if lv in ("(10) 10-Certain will vote", "(12) Already voted", "(8) 8", "(9) 9"):
  likelyvoter = "(1) Likely voter"
else:
  likelyvoter = "(2) Nonvoter"

#@m

## Form

In [0]:
#@title Propensity Estimate of Trump Support
#@markdown 1. How are you completing this survey?
mode = "(1) Phone" #@param ["(1) Phone", "(2) Web"]

#@markdown 2. How interested are you in the election taking place on November 6th, 2018?
lva = "(1) Extremely interested" #@param ["(1) Extremely interested", "(2) Very interested", "(3) Somewhat interested", "(4) Only a little interested", "(5) Not at all interested"]

#@markdown 3. On a scale from zero to 10, where 10 means you're certain you will vote and zero means there is no chance you will vote, please indicate how likely it is that you will vote in the 2018 midterm election.
lv = "(0) 0-Certain will not vote" #@param ["(0) 0-Certain will not vote", "(1) 1", "(2) 2", "(3) 3", "(4) 4", "(5) 5", "(6) 6", "(7) 7", "(8) 8", "(9) 9", "(10) 10-Certain will vote", "(12) Already voted"]

if lv in ("(10) 10-Certain will vote", "(12) Already voted", "(8) 8", "(9) 9"):
  likelyvoter = "(1) Likely voter"
else:
  likelyvoter = "(2) Nonvoter"

#@markdown 4. In 2018, did you vote at a polling place on Election Day, or did you vote by mail or vote in person at some point before Election Day?
timevote = "(1) Election Day" #@param ["(1) Election Day", "(2) Early/Absentee"]

#@markdown 5. Generally speaking, would you say things in this country are heading in the…
track = "(1) Right direction" #@param ["(1) Right direction", "(2) Wrong direction"]

#@markdown 6. Which one of the following would you say is the most important issue facing the country?
issues = "(1) The economy and jobs" #@param ["(1) The economy and jobs", "(2) Health care", "(3) Immigration", "(4) Taxes", "(5) Abortion", "(6) Gun policy", "(7) The environment", "(8) Foreign policy", "(9) Terrorism"]
issues_economyandjobs = 0
issues_healthcare = 0
issues_immigration = 0
issues_taxes = 0
issues_abortion = 0
issues_gunpolicy = 0
issues_environment = 0
issues_foreignpolicy = 0
if issues == "(1) The economy and jobs":
  issues_economyandjobs = 1
elif issues == "(2) Health care":
  issues_healthcare = 1
elif issues == "(3) Immigration":
  issues_immigration = 1
elif issues == "(4) Taxes":
  issues_taxes = 1
elif issues == "(5) Abortion":
  issues_abortion = 1
elif issues == "(6) Gun policy":
  issues_gunpolicy = 1
elif issues == "(7) The environment":
  issues_environment = 1
elif issues == "(8) Foreign policy":
  issues_foreignpolicy = 1

#@markdown 7. Which best describes your family's financial situation? Are you…
getahead = "(1) Getting ahead" #@param ["(1) Getting ahead", "(2) Holding steady", "(3) Falling behind"]

#@markdown 8. Do you think the Trump administration's trade policies will help or hurt the national economy, or won't they make much difference?
tradenationalecon = "(1) Help" #@param ["(1) Help", "(2) Hurt", "(3) No difference"]

#@markdown 9. Do you think the Trump administration's trade policies will help or hurt the economy in your area, or won't they make much difference?
tradelocalecon = "(1) Help" #@param ["(1) Help", "(2) Hurt", "(3) No difference"]

#@markdown 10. Which of the following comes closest to what you would like to see lawmakers do with the Affordable Care Act, also known as Obamacare?
healthlaw = "(1) Repeal the law entirely" #@param ["(1) Repeal the law entirely", "(2) Repeal parts of the law", "(3) Leave the law as is", "(4) Expand the law"]

#@markdown 11. Which of the following comes closer to your opinion? Should most immigrants who are living in the United States illegally be…
immdeport = "(1) Offered a chance to apply for legal status" #@param ["(1) Offered a chance to apply for legal status", "(2) Deported to the country they came from"]

#@markdown 12. Do you think the Trump campaign coordinated with Russia during the 2016 presidential election, or not?
russia = "(1) Yes" #@param ["(1) Yes", "(2) No"]

#@markdown 13. Regarding the 2014 election for Congress, which one of the following statements best describes you?
qpvvote3 = "(1) I did not vote in the 2014 election for Congress" #@param ["(1) I did not vote in the 2014 election for Congress", "(2) I thought about voting in the 2014 election for Congress, but didn't", "(3) I usually vote, but I didn't in the 2014 election for Congress", "(4) I'm sure I voted"]

#@markdown 14. Do you think the way Democrats talk about politics these days is leading to an increase in acts of violence, or don't you think so?
breaka = "(1) Yes, it is" #@param ["(1) Yes, it is", "(2) No, it is not"]

#@markdown 15. Do you think the way Republicans talk about politics these days is leading to an increase in acts of violence, or don't you think so?
breakb = "(1) Yes, it is" #@param ["(1) Yes, it is", "(2) No, it is not"]

#@markdown 16. Are you male or female?
sex = "(1) Men" #@param ["(1) Men", "(2) Women"]

#@markdown 17. What is your age?
age65 = "(1) 18-24" #@param ["(1) 18-24", "(2) 25-29", "(3) 30-39", "(4) 40-49", "(5) 50-64", "(6) 65+"]
age_18_24 = 0
age_25_29 = 0
age_30_39 = 0
age_40_49 = 0
age_50_64 = 0
if age65 == "(1) 18-24":
  age_18_24 = 1
elif age65 == "(2) 25-29":
  age_25_29 = 1
elif age65 == "(3) 30-39":
  age_30_39 = 1
elif age65 == "(4) 40-49":
  age_40_49 = 1
elif age65 == "(5) 50-64":
  age_50_64 = 1

#@markdown 18. What is your racial or ethnic heritage?
raceth = "(1) White" #@param ["(1) White", "(2) African American or Black", "(3) Latino or Hispanic", "(4) Asian", "(5) American Indian, Native American or Alaska Native", "(6) Native Hawaiian or Other Pacific Islander", "(7) Other"]
raceth_wht = 0
raceth_blk = 0
raceth_hsp = 0
raceth_asn = 0
raceth_ntvam = 0
raceth_hipac = 0
if raceth == "(1) White":
  raceth_wht = 1
elif raceth == "(2) African American or Black":
  raceth_blk = 1
elif raceth == "(3) Latino or Hispanic":
  raceth_hsp = 1
elif raceth == "(4) Asian":
  raceth_asn = 1
elif raceth == "(5) American Indian, Native American or Alaska Native":
  raceth_ntvam = 1
elif raceth == "(6) Native Hawaiian or Other Pacific Islander":
  raceth_hipac = 1

raceth5 = None

#@markdown 19. Which best describes your level of education?
educ = "(1) High school or less" #@param ["(1) High school or less", "(2) Some college/assoc. degree", "(3) College graduate", "(4) Postgraduate study"]
edu_highschoolorless = 0
edu_somecollege_assocdegree = 0
edu_collegegraduate = 0
if educ == "(1) High school or less":
  edu_highschoolorless = 1
elif educ == "(2) Some college/assoc. degree":
  edu_somecollege_assocdegree = 1
elif educ == "(3) College graduate":
  edu_collegegraduate = 1

#@markdown 20. In which of these categories is your total yearly household income, that is, of everyone living in your family household? Was your total household income in 2017…
income = "(1) Under $25,000" #@param ["(1) Under $25,000", "(2) $25,000-$49,999", "(3) $50,000-$74,999", "(4) $75,000-$99,999", "(5) $100,000 or more"]
income_under25000 = 0
income_25000_49999 = 0
income_50000_74999 = 0
income_75000_99999 = 0
if income == "(1) Under $25,000":
  income_under25000 = 1
elif income == "(2) $25,000-$49,999":
  income_25000_49999 = 1
elif income == "(3) $50,000-$74,999":
  income_50000_74999 = 1
elif income == "(4) $75,000-$99,999":
  income_75000_99999 = 1

#@markdown 21. Generally speaking, do you consider yourself a Democrat, a Republican, or don't you consider yourself either?
party = "(1) Democrat" #@param ["(1) Democrat", "(2) Republican", "(3) Neither"]

#@markdown 22. Would you say you're closer to the Democrats, to the Republicans, or don't you lean toward either?
partyfull = "(1) Democrat/Lean Dem" #@param ["(1) Democrat/Lean Dem", "(2) Republican/Lean Rep", "(3) Independent"]

#@markdown 23. On most political matters, do you consider yourself…
ideol = "(1) Very liberal" #@param ["(1) Very liberal", "(2) Somewhat liberal", "(3) Moderate", "(4) Somewhat conservative", "(5) Very conservative"]

#@markdown 24. What is your religion, if any?
relig = "(1) Protestant" #@param ["(1) Protestant", "(2) Catholic", "(3) Mormon", "(4) Other Christian", "(5) Jewish", "(6) Muslim", "(7) Something else", "(8) None"]
relig_prot_chrst = 0
relig_catholic = 0
relig_mormon = 0
relig_jewish = 0
relig_muslim = 0
relig_none = 0
if relig == "(1) Protestant" or relig == "(4) Other Christian":
  relig_prot_chrst = 1
elif relig == "(2) Catholic":
  relig_catholic = 1
elif relig == "(3) Mormon":
  relig_mormon = 1
elif relig == "(5) Jewish":
  relig_jewish = 1
elif relig == "(6) Muslim":
  relig_muslim = 1
elif relig == "(8) None":
  relig_none = 1

relig4 = None

#@markdown 25. Which of the following best describes the area where you live?
sizeplace = "(1) Urban" #@param ["(1) Urban", "(2) Suburban", "(3) Small town", "(4) Rural"]
sizeplace_urban = 0
sizeplace_suburban = 0
sizeplace_smalltown = 0
if sizeplace == "(1) Urban":
  sizeplace_urban = 1
elif sizeplace == "(2) Suburban":
  sizeplace_suburban = 1
elif sizeplace == "(3) Small town":
  sizeplace_smalltown = 1

print('Inputs saved!')
ui_input = pd.DataFrame([[mode,lva,lv,likelyvoter,timevote,track,issues_economyandjobs,issues_healthcare,issues_immigration,issues_taxes,issues_abortion,issues_gunpolicy,issues_environment,issues_foreignpolicy,getahead,tradenationalecon,tradelocalecon,healthlaw,immdeport,russia,qpvvote3,breaka,breakb,sex,age_18_24,age_25_29,age_30_39,age_40_49,age_50_64,raceth_wht,raceth_blk,raceth_hsp,raceth_asn,raceth_ntvam,raceth_hipac,edu_highschoolorless,edu_somecollege_assocdegree,edu_collegegraduate,income_under25000,income_25000_49999,income_50000_74999,income_75000_99999,party,partyfull,ideol,relig_prot_chrst,relig_catholic,relig_mormon,relig_jewish,relig_muslim,relig_none,sizeplace_urban,sizeplace_suburban,sizeplace_smalltown]],columns=('mode','lva','lv','likelyvoter','timevote','track','issues_economyandjobs','issues_healthcare','issues_immigration','issues_taxes','issues_abortion','issues_gunpolicy','issues_environment','issues_foreignpolicy','getahead','tradenationalecon','tradelocalecon','healthlaw','immdeport','russia','qpvvote3','breaka','breakb','sex','age_18_24','age_25_29','age_30_39','age_40_49','age_50_64','raceth_wht','raceth_blk','raceth_hsp','raceth_asn','raceth_ntvam','raceth_hipac','edu_highschoolorless','edu_somecollege_assocdegree','edu_collegegraduate','income_under25000','income_25000_49999','income_50000_74999','income_75000_99999','party','partyfull','ideo','relig_prot_chrst','relig_catholic','relig_mormon','relig_jewish','relig_muslim','relig_none','sizeplace_urban','sizeplace_suburban','sizeplace_smalltown'))
ui_input = call_to_binary(ui_input,'mode',val_1=2)
ui_input = call_to_ranked_reverse(ui_input,'lva','electinterest',5)
ui_input = call_to_prob(ui_input,'lv','P_vote')
ui_input = call_to_binary(ui_input,'likelyvoter')
ui_input = call_to_binary(ui_input,'timevote')
ui_input = call_to_binary(ui_input,'track')
ui_input = call_to_ranked_reverse(ui_input,'getahead','famfinancial',3)
ui_input = call_to_3_choice_flip(ui_input,'tradenationalecon','trumpnatecon')
ui_input = call_to_3_choice_flip(ui_input,'tradelocalecon','trumplocalecon')
ui_input = call_to_ranked_reverse(ui_input,'healthlaw','supACA',4)
ui_input = call_to_binary(ui_input,'immdeport','immpol')
ui_input = call_to_binary(ui_input,'russia','russiacollude16')
ui_input = call_to_binary(ui_input,'qpvvote3','voted14',4)
ui_input = call_to_binary(ui_input,'breaka','violencespeechD')
ui_input = call_to_binary(ui_input,'breakb','violencespeechR')
ui_input = call_to_binary(ui_input,'sex','sex_m')
ui_input = call_combo_party(ui_input)
ui_input = call_ideo(ui_input)
pred = logmodel.predict_proba(ui_input)
print("P(suppTrump=0) P(supTrump=1)")
print(pred)